In [1]:
from goose3 import Goose
g = Goose({'enable_image_fetching':False, 'browser_user_agent': 'Mozilla'})
def extract_text(article_url):
    article = g.extract(url=article_url)
    return article

In [2]:
from urllib.parse import unquote

prefix_blacklist = ['https://www.google.com', 'https://www.google.com/search?q=', 'https://support.google.com', 'https://www.google.com/intl/en/about/products', 'https://accounts.google.com', 'https://maps.google.com/maps', 'https://www.google.com/flights', 'https://www.google.com/preferences', 'https://www.google.com/setprefs', 'https://www.google.com/history', 'https://www.google.com/alerts', 'https://newsinitiative.withgoogle.com', 'https://policies.google.com', 'https://newsinitiative.withgoogle.com']

def check_prefixes(link, blacklist=prefix_blacklist):
    for p in blacklist:
        if link[:len(p)] == p:
            return False
    return True

def convert_gamp(link, prefix='https://www.google.com/url'):
    if link[:len(prefix)] == prefix:
        return unquote(link.split('&url=')[1])
    return link

In [3]:
import pandas as pd
import os
from glob import glob
from tqdm.notebook import tqdm
import pickle

In [4]:
urls = set()
for csv_path in glob('scraped/*/FINAL.csv'):
    new_urls = pd.read_csv(csv_path)['url'].to_list()
    urls.update(new_urls)

In [5]:
raw_urls = list(urls)

In [6]:
urls = list(map(convert_gamp, raw_urls))
urls = list(filter(check_prefixes, urls))

In [8]:
len(urls)

4559

In [9]:
num_skipped = 0
text_objects = []

for u in tqdm(urls):
    try:
        article_goose = extract_text(u)
        text_objects.append(article_goose.infos)
    except Exception as e:
        print(e)
        num_skipped += 1

print('skipped %d' % num_skipped)

print('saving...')
with open('article_objects.pkl', 'wb') as file_out:
    pickle.dump(text_objects, file_out)

c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname PT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
c:\users\alex\anaconda3\envs\scrape_news\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
HTTPSConnectionPool(host='charlottenc.gov', port=443): Max retries exceeded with url: /covid19/Pages/default.aspx (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)')))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnect

In [15]:
len(text_objects)

4467

In [18]:
no_empty = []
for to in text_objects:
    if not to['cleaned_text'] == '':
        no_empty.append(to)
text_objects = no_empty

In [19]:
print('saving...')
with open('article_objects_noempty.pkl', 'wb') as file_out:
    pickle.dump(text_objects, file_out)

saving...


In [20]:
print('loading...')
with open('article_objects_noempty.pkl', 'rb') as file_in:
    text_objects = pickle.load(file_in)

loading...


In [21]:
df = pd.DataFrame(text_objects)

In [22]:
text_data = df['cleaned_text'].to_list()

In [152]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [153]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from unidecode import unidecode
import re

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
cachedStopWords = stopwords.words("english")

def preprocess(raw):
    result = unidecode(raw)
    result = re.sub(r'(https?:\/\/)?([a-zA-Z0-9-_]{1,}\.){1,}[a-zA-Z0-9-_]{1,}(\/[A-Za-z0-9-._~:?#\[\]@!$&\'()*+,;%=]{1,}){0,}\/?', '', result)
    result = re.sub(r"[^a-zA-Z]", " ", result)
    result = re.sub(' +', ' ', result)
    result = result.lower()
    result = stemmer.stem(result)
    result = [lemmatiser.lemmatize(w) for w in result.split(' ')]
    result = [w for w in result if w not in cachedStopWords]
    return result

tp = []
for article in tqdm(text_data):
    tp.append(preprocess(article))

In [154]:
with open('processed_text.pkl', 'wb') as file_out:
    pickle.dump(tp, file_out)